#### First Implementation of scraping all events of defined Website Scrapers and Visualize them in Qgrid

In [ ]:
import datetime as dt
import qgrid
from src import scraper_collection

In [ ]:
end_date= dt.datetime(2023,8,6)
print(end_date)
events = scraper_collection.get_all_events_df(end_date)
events

In [ ]:
qgrid_widget = qgrid.show_grid(events[["start_date", "event_type", "title", "url", "organiser", "location", "periodic"]], show_toolbar=True)
qgrid_widget